In [ ]:
import commpy
import numpy as np
import scipy.interpolate 
#import tensorflow as tf
import math
import matplotlib.pyplot as plt
import os

#### Steps to simulate a coounication system

1. Bit Stream
1. Symbol Mapper
1. Transmitter
1. Channel
1. Estimator
1. Decoder

#### Things to be decided and fixed across all the simulations :

1. Bandwidth of the signal to be used: From Andrea Goldsmith example of 20MHz is used for all the subcarriers combines. We can use the same.

1. Frequency range of operation : (?)

In [ ]:
K = 64  # Number of carriers
CP = 16 # Cyclic Prefix
P = 4 # Carriers used for Pilot Symbols used
# Total of 48 sub carriers are available for data symbols

In [ ]:
def gen_bits(size):
    return np.random.randint(2,size=(size,))

In [ ]:
def OFDM_IFFT(symbol):
    return np.fft.ifft(symbol)
def OFDM_FFT(symbol):
    return np.fft.fft(symbol)
def addCP(x, len_cp):
    cp= x[-len_cp:]
    return np.append(cp,x)

In [ ]:
data_bits=gen_bits(48*2)  # Genrating the data bits to map it onto 48 subcarrier using QPSK
zero_bits = np.zeros((12*2,), dtype=int)  # Zero bits for inter-channel interference, something A.Goldsmith mentions
pilot_bits = np.zeros((4*2,), dtype = int) # Generating Pilot based bits
bit_frame = np.concatenate((pilot_bits,zero_bits,data_bits)) # One complete bit frame 

In [ ]:
QAM = commpy.QAMModem(4) # Creating QAM object for 4 symbol map i.e. 2 bits mapped onto each symbol
X=QAM.modulate(bit_frame) # Creating the Symbols from the bits, len = 64

In [ ]:
## Generating the OFDM frame with CP ###
x = OFDM_IFFT(X)
OFDM_frame= addCP(x,len_cp=16)

### Creating the Channel 

#### Two main things are required to create the channell array :

1. Type of Small scale fading model i.e. Rayleigh (NLOS), Rician (LOS) or Nakagami (NLOS + LOS)
1. Power Delay Profile ( Default taken as exponential)

##### Creating the channel array 

1. The length of the channel array depends on the value of the maximum delay of the channel i.e. if the maximum delay is 8*$ T_{s}$, where $T_{s}$ is the sampling frequency, then the channel length will be of 9 elements. To decide the maximum delay IEEE 802.11 model tells us
$$n_{max} = [10*\frac{\sigma_{\tau}}{T_{s}}] $$
where $\sigma_{\tau}$ is the delay spread.

1. The power of the channel taps are given by the exponential distribution for the 802.11 model. 

Note : The number of channel taps is not the length of the channel array. The length is completely dependant on the maximum delay value, the intermediate values between the channel taps are padded with zeroes.


In [ ]:
### Example of channel array taken from the MIMO-OFDM cho book ###

delay = np.array([0, 3, 5, 6, 8])  # Number of taps taken as five
power_dB = np.array([0, -8, -17, -21, -25]) # Comparative power values wrt no dela signal, exponential distribution
taps = len(delay)
l_h = delay[-1]+1 # Length of channel array 

h_taps = (np.random.rand(1,taps))+ 1j*np.random.rand(1,taps)*np.power(np.power(10,power_dB/10)/2,0.5) # Creating the channel 
#taps for rayleigh fading with the given power profile, the divided by 2 is for each real and imaginary part.
h= np.zeros(l_h ,dtype = 'complex') # Initializing the channel time domain array

h[delay]=h_taps

In [25]:
def channel_output(x,h,SNR,ch_type=1):
    
    '''
    Provides the output of the channel after the signal has passed through it
    x: Input signal
    h: Channel vector
    ch_type : Flat fading channel (0) or Frequency Selective Channel (1)
    SNR : To determine the output noise power
    '''
    
    if (ch_type == 0):
        y= h*x
    else :
        y= np.convolve(h,x)
    
    sig_pow = np.sum(y**2)
    noise_mag = np.power(np.power(10,(-1*SNR)/10) * sig_pow/2,0.5)
    noise = (np.random.rand(1,len(y)) + 1j*np.random.rand(1,len(y))) * noise_mag
    
    return y + noise

### Cooperative OFDM 

1. Source
1. Relay
1. Destination

##### Protocol used for sending information

1. MultiHop (MxF) : First time slot S->R, second time slot R->D
1. Split Combine : in the first timeslot, the source transmits and only the relay is listening (just as in MxF). In the second timeslot, both source and relay transmit, and the destination is listening.
1. Diversity : in the first timeslot, the source transmits and both relay and destination are listening. In the second timeslot, only the relay is transmitting and the destination is listening. Thus, the destination gets two copies of the original signal.

##### Different Relay Methods we can use

1. Ampilify and Forward : The signal from the source node is received which is amplified by the following factor and then forwarded to the destination.

$$ A = \sqrt\frac{P_{R}}{P_{S}*h_{SR}^2 + P_{N}}  $$

The amplification taken is the limiting maximum value. 